# IMDB Sentiment Analyses¶
Neste notebook estamos utilizando os dados do Kaggle (https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-ovie-reviews).

Vamos seguir os seguintes passos:

1. Importar o Dataset
2. Preparar os Dados para Construir o Modelo
3. Criar o Dataset de Teste e Treino
4. Treino do Modelo Utilizando Keras
5. Salva o modelo

In [ ]:
import pandas as pd
import numpy as np
import pickle
import nltk
import keras

from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing

# 1 - Importar o Dataset

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

stop_words = stopwords.words('english')
porter_stemmer = PorterStemmer()

df = pd.read_csv('./imdb-dataset.csv', delimiter=',')
df = df.head(30000)

# 2 - Preparar os Dados para Construir o Modelo
Agora vamos preparar nossos seguindo os passos:

1. Tonkenization         
2. Remover stopwords
3. Stemming text
4. Juntar novamente em uma única frase

Como estamos trabalhando com uma entrada de texto, realizamos estas etapas para "normalizar" nossa base.

In [ ]:
def identify_tokens(row):
    source = row[0]
    tokens = word_tokenize(source)
    token_words = [w for w in tokens if w.isalpha()]
    return token_words

In [ ]:
def remove_stops(row):
    source_tokenization = row[2]
    stop = [w for w in source_tokenization if not w in stop_words]
    return (stop)

In [ ]:
def stem_porter(row):
    my_list = row[2]
    stemmed_list = [porter_stemmer.stem(word) for word in my_list]
    return (stemmed_list)

In [ ]:
def rejoin_words(row):
    my_list = row[2]
    joined_words = (" ".join(my_list))
    return joined_words

In [ ]:
def pre_processing(df):
    print('Tokenization')
    df['text1'] = df.apply(identify_tokens, axis=1)
    print('Remove stop words')
    df['text1'] = df.apply(remove_stops, axis=1)
    print('Stemming')
    df['text1'] = df.apply(stem_porter, axis=1)
    print('Rejoin words')
    df['tidy_text'] = df.apply(rejoin_words, axis=1)

    return df

In [ ]:
df = pre_processing(df)
df['tidy_text'] = df['tidy_text'].str.lower()

X = df['tidy_text']
Y = df['sentiment']

# 3 - Criar o Dataset de Teste e Treino¶
Vamos criar o nosso dataset de teste (30%) e treino (70%) de forma balanceado (Stratified)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,
                                                    Y,
                                                    test_size=0.1,
                                                    random_state=48,
                                                    stratify=Y)

vectorizer = TfidfVectorizer(ngram_range=(2, 3),
                        sublinear_tf=True,
                        max_features=10000)

Os modelos de Machine Learning ou Deep Learning esperam como entrada "X" um valor numérico. Como estamos trabalhando com texto iremos realizar o processo de TfIdf para transformar o texto em valores numéricos.

In [ ]:
X_train_tf = vectorizer.fit_transform(X_train)
X_test_tf = vectorizer.transform(X_test)

le = preprocessing.LabelEncoder()

le.fit(list(Y_train))
Y_train_le = le.transform(list(Y_train))
Y_test_le = le.transform(list(Y_test))

num_class = Y.value_counts().shape
input_shape = X_train_tf.shape

# 4 - Treino do Modelo Utilizando Keras

In [ ]:
from keras.utils import to_categorical

Y_train_label_keras = to_categorical(Y_train_le)
Y_test_label_keras = to_categorical(Y_test_le)

from keras import models
from keras import layers

In [ ]:
network = models.Sequential()

network.add(layers.Dense(2, activation='relu', input_shape=(input_shape[1], )))
network.add(layers.Dropout(0.4))

network.add(layers.Dense(5, activation='relu'))
network.add(keras.layers.Dropout(0.4))

network.add(layers.Dense(5, activation='sigmoid'))
network.add(layers.Dropout(0.4))

network.add(layers.Dense(num_class[0], activation='softmax'))

network.compile(optimizer='adamax',
                loss="binary_crossentropy",
                metrics=['accuracy'])

network.summary()

In [ ]:
network.fit(X_train_tf.toarray(),
            Y_train_label_keras,
            verbose=1,
            epochs=50,
            validation_split=0.3)

# 5 - Salva o Modelo

In [ ]:
network.save('./model/neural_network.h5')
pickle.dump(vectorizer, open('./model/vectorizer.pkl', 'wb'))